In [3]:
from pyscf import gto, scf
from scipy.linalg import fractional_matrix_power
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

class RHF:
    def __init__(self, atoms, basis='ccpvdz'):
        self.atoms = atoms
        self.basis = basis
        self.mol = gto.M(atom=self.atoms, basis=self.basis)
        self.S = self.mol.intor('int1e_ovlp')
        self.T = self.mol.intor('int1e_kin')
        self.V = self.mol.intor('int1e_nuc')
        self.H_core = self.T + self.V
        self.eri = self.mol.intor('int2e')
        self.enuc = self.mol.get_enuc()
        self.ndocc = self.mol.nelec[0]
        self.C = None
        self.SCF_E = None

    def rhf(self):
        A = fractional_matrix_power(self.S, -0.5)
        A = np.asarray(A)
        F_p = A @ self.H_core @ A
        epsilon, C_p = np.linalg.eigh(F_p)
        self.C = A @ C_p
        C_occ = self.C[:, :self.ndocc]
        D = np.einsum('ik,kj->ij', C_occ, C_occ.T, optimize=True)

        SCF_E = 0.0
        E_old = 0.0

        MAXITER = 100000
        E_conv = 1.0e-6

        for scf_iter in range(1, MAXITER + 1):
            J = np.einsum('pqrs,rs->pq', self.eri, D, optimize=True)
            K = np.einsum('prqs,rs->pq', self.eri, D, optimize=True)

            F = self.H_core + 2 * J - K
            SCF_E = self.enuc + np.sum(D * (self.H_core + F))

            if abs(SCF_E - E_old) < E_conv:
                break
            E_old = SCF_E

            epsilon, C_p = np.linalg.eigh(A @ F @ A)
            self.C = A @ C_p
            C_occ = self.C[:, :self.ndocc]
            D = np.einsum('pi,iq->pq', C_occ, C_occ.T, optimize=True)

            if scf_iter == MAXITER:
                raise Exception("Maximum number of SCF iterations exceeded.")
        self.SCF_E = SCF_E

benzene_geometry = '''
   C        0.00000        1.39767        0.00000
   C        1.20912        0.69884        0.00000
   C        1.20912       -0.69884        0.00000
   C        0.00000       -1.39767        0.00000
   C       -1.20912       -0.69884        0.00000
   C       -1.20912        0.69884        0.00000
   H        0.00000        2.49029        0.00000
   H        2.15741        1.24515        0.00000
   H        2.15741       -1.24515        0.00000
   H        0.00000       -2.49029        0.00000
   H       -2.15741       -1.24515        0.00000
   H       -2.15741        1.24515        0.00000
'''

benzene_molecule = RHF(benzene_geometry, basis='ccpvdz')
benzene_molecule.rhf()

print("Overlap matrix (S):")
print(benzene_molecule.S)

print("\nAO labels:")
print(benzene_molecule.mol.ao_labels())

print("\nMolecular orbital coefficients (C):")
print(benzene_molecule.C)

print("\nFinal RHF Energy:")
print(benzene_molecule.SCF_E)


Overlap matrix (S):
[[ 1.00000000e+00  2.48362390e-01  0.00000000e+00 ...  1.22561022e-05
   9.24764894e-05  5.32602902e-03]
 [ 2.48362390e-01  1.00000000e+00  0.00000000e+00 ...  1.85435892e-03
   6.47060770e-03  9.60494232e-02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00 ...  0.00000000e+00
  -6.78239172e-03 -1.23808921e-01]
 ...
 [ 1.22561022e-05  1.85435892e-03  0.00000000e+00 ...  1.00000000e+00
   5.03662033e-02  7.76052834e-04]
 [ 9.24764894e-05  6.47060770e-03 -6.78239172e-03 ...  5.03662033e-02
   1.00000000e+00  5.04267473e-02]
 [ 5.32602902e-03  9.60494232e-02 -1.23808921e-01 ...  7.76052834e-04
   5.04267473e-02  1.00000000e+00]]

AO labels:
['0 C 1s    ', '0 C 2s    ', '0 C 2px   ', '0 C 2py   ', '0 C 2pz   ', '1 C 1s    ', '1 C 2s    ', '1 C 2px   ', '1 C 2py   ', '1 C 2pz   ', '2 C 1s    ', '2 C 2s    ', '2 C 2px   ', '2 C 2py   ', '2 C 2pz   ', '3 C 1s    ', '3 C 2s    ', '3 C 2px   ', '3 C 2py   ', '3 C 2pz   ', '4 C 1s    ', '4 C 2s    ', '4 C 2px   ', '4 C 2py 